In [1]:
import pandas as pd
import altair as alt
from altair.expr import datum
import warnings
from altair_transform import extract_data
from altair_transform import transform_chart
warnings.filterwarnings('ignore')

In [2]:
### Load the dataframe to be visualized 
Indiv_melted_filename = '../Saved/4vis/Indiv_melted.pkl'
Indiv_melted = pd.read_pickle(Indiv_melted_filename)

# load Model Prediction
predictProb_filename = '../Saved/Attr/lime_infos.pkl'
predict_prob_df = pd.DataFrame(data = pd.read_pickle(predictProb_filename)['model_pred'], columns=['model_pred'])

In [3]:
dict_method_name_to_attr_name = {
    'shap' : 'shap_value',
    'lime': 'lime_weight', 
    'ig':  'ig_attr', 
    'deepLift':'deepLift_attr' 
}

## Make the Charts

In [4]:
def feature_attr_and_rank_plot(index):
    values_melted = Indiv_melted.loc[index]
    sorted_feature = values_melted[values_melted['method'] == 'sum_rank'].sort_values(by = 'feature_attr', ascending = False)['feature_name'].to_list() 
    values_melted['is_positive'] = values_melted['feature_attr'] > 0

    ## bar chart for feature attribution values 
    brush = alt.selection_multi(on='mouseover', toggle=True, fields=['feature_name'])
    base = alt.Chart(values_melted).mark_bar().encode(
        opacity = alt.condition(brush, alt.value(1.0), alt.value(0.2))
    ).add_selection(
        brush
    )

    row1 = alt.hconcat()
    row2 = alt.hconcat()
    i = 0
    for _, attr_name in dict_method_name_to_attr_name.items():
        chart = base.encode(
            y=alt.Y('feature_name', sort='-x', title = ''),
            x= alt.X('feature_attr', title = attr_name),
            color = alt.Color('is_positive', legend=None, scale=alt.Scale(range=['crimson', 'steelblue']))
        ).transform_filter(
            datum.method == attr_name
        ).properties(
            width=200,
            height=200
        )
        if i < 2:
            row1 |= chart
        else:
            row2 |= chart
        i += 1
    attr_value_bar = alt.vconcat(row1, row2)
 
    model_predict = round(predict_prob_df.iloc[index].values[0], 4)
    attr_value_bar = alt.vconcat(row1, row2)

    # bar chart for signed feature rank: group lollipop chart visualization 
    line = base.mark_bar(size = 5).encode(
        x = alt.X('method', title=''),
        y = alt.X('feature_attr', title = 'feature_rank'),
        color = alt.Color('is_positive', legend=None, scale=alt.Scale(range=['crimson', 'steelblue'])) 
    ).transform_filter(
        alt.FieldOneOfPredicate(field='method', oneOf= ['shap_rank', 'lime_rank', 'ig_rank', 'deepLift_rank'])
    ).properties(
        width=50,
        height=200
    )
    attr_signed_rank = (line).facet(column=alt.Column('feature_name', sort = sorted_feature)) 

    return alt.hconcat(attr_value_bar, attr_signed_rank, title = 'model predict: ' + str(model_predict))

In [6]:
# Show the visualization 
data_index = 400
feature_attr_and_rank_plot(data_index)

alt.HConcatChart(...)